In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from KL_PMVAE import KL_PMVAE_Cancer
from utils import sort_data, load_data, bce_recon_loss, kl_divergence, get_match_id
from train_KL_PMVAE import train_KL_PMVAE
dtype = torch.FloatTensor

# Parameters specific to your dataset
input_n1 = 810  # Update based on your radiomic features
input_n2 = 18   # Update based on your clinical features
z_dim = [8, 16, 32, 64]
EPOCH_NUM = [800, 1200, 1600]
NUM_CYCLES = [2, 4, 5]
CUTTING_RATIO = [0.3, 0.5, 0.7]
Initial_Learning_Rate = [0.05, 0.005, 0.001]
L2_Lambda = [0.1, 0.05, 0.005]

# Load training and validation data
_, x_train_radiomic, ytime_train, yevent_train = load_data("train_clinical.csv", dtype)
_, x_valid_radiomic, ytime_valid, yevent_valid = load_data("test_clinical.csv", dtype)

# Hyperparameter tuning
opt_l2 = 0
opt_lr = 0
opt_dim = 0
opt_epoch_num = 0
opt_num_cycle = 0
opt_cr = 0
opt_loss = torch.Tensor([float("Inf")])
if torch.cuda.is_available():
    opt_loss = opt_loss.cuda()

for l2 in L2_Lambda:
    for lr in Initial_Learning_Rate:
        for Z in z_dim:
            for Epoch_num in EPOCH_NUM:
                for Num_cycles in NUM_CYCLES:
                    for cutting_ratio in CUTTING_RATIO:
                        _, _, _, _, train_loss_unsup, eval_loss_unsup = train_KL_PMVAE(
                            x_train_radiomic, yevent_train,
                            x_valid_radiomic, yevent_valid,
                            Z, input_n1, input_n2,
                            lr, l2, cutting_ratio, Epoch_num, Num_cycles, dtype,
                            path="saved_models/unsup_checkpoint_tune.pt"
                        )
                        if eval_loss_unsup < opt_loss:
                            opt_l2 = l2
                            opt_lr = lr
                            opt_dim = Z
                            opt_epoch_num = Epoch_num
                            opt_num_cycle = Num_cycles
                            opt_cr = cutting_ratio
                            opt_loss = eval_loss_unsup
                        print(f"Epoch: {Epoch_num}, Cycles: {Num_cycles}, CR: {cutting_ratio}, "
                              f"L2: {l2}, LR: {lr}, z_dim: {Z}, "
                              f"Train Loss: {train_loss_unsup:.4f}, Eval Loss: {eval_loss_unsup:.4f}")

print(f"Optimal Epochs: {opt_epoch_num}, Cycles: {opt_num_cycle}, CR: {opt_cr}, "
      f"L2: {opt_l2}, LR: {opt_lr}, z_dim: {opt_dim}")

# Final training with optimal parameters
train_mean, train_logvar, test_mean, test_logvar, train_loss_unsup, test_loss_unsup = train_KL_PMVAE(
    x_train_radiomic, yevent_train,
    x_valid_radiomic, yevent_valid,
    opt_dim, input_n1, input_n2,
    opt_lr, opt_l2, opt_cr, opt_epoch_num, opt_num_cycle, dtype, save_model=True,
    path="saved_models/unsup_checkpoint_final.pt"
)
print(f"Final Training Loss: {train_loss_unsup:.4f}, Testing Loss: {test_loss_unsup:.4f}")

# Process and save latent representations
train_z = train_mean
test_z = test_mean

processed_train = torch.cat((ytime_train, yevent_train, train_z), 1)
processed_test = torch.cat((ytime_valid, yevent_valid, test_z), 1)

z_count = np.array(list(range(1, train_z.size()[1] + 1))).astype(str)
z_names = [f"Z_{i}" for i in z_count]

processed_train_df = pd.DataFrame(processed_train.cpu().numpy(), columns=["PFS", "Censorship"] + z_names)
processed_test_df = pd.DataFrame(processed_test.cpu().numpy(), columns=["PFS", "Censorship"] + z_names)

processed_train_df.to_csv("processed_train.csv", index=False)
processed_test_df.to_csv("processed_test.csv", index=False)

Index(['SubjectID', 'Sex', 'Age at Diagnosis', 'Extent of Tumor Resection',
       'Chemotherapy', 'Radiation', 'Censorship', 'NF1',
       'Progression Free Survival', 'Tumor_Location_Basal Ganglia',
       'Tumor_Location_Brainstem', 'Tumor_Location_Cerebellar',
       'Tumor_Location_Lobar', 'Tumor_Location_Multifocal',
       'Tumor_Location_OPG', 'Tumor_Location_Suprasellar',
       'Tumor_Location_Tectum', 'Tumor_Location_Thalamus',
       'Tumor_Location_Ventricular'],
      dtype='object')
Index(['SubjectID', 'Sex', 'Age at Diagnosis', 'Extent of Tumor Resection',
       'Chemotherapy', 'Radiation', 'Censorship', 'NF1',
       'Progression Free Survival', 'Tumor_Location_Basal Ganglia',
       'Tumor_Location_Brainstem', 'Tumor_Location_Cerebellar',
       'Tumor_Location_Lobar', 'Tumor_Location_Multifocal',
       'Tumor_Location_OPG', 'Tumor_Location_Suprasellar',
       'Tumor_Location_Tectum', 'Tumor_Location_Thalamus',
       'Tumor_Location_Ventricular'],
      dtype='obj

NameError: name 'torch' is not defined